## Shopping cart in Redis

Why not cookie?

- cookies needs to be parsed and validated to ensure it has the proper format and contain items that can actually be purchased
- cookies are passed down through every request, which can slow down request when the cookie is large

We can use Redis HASHes to store the mapping of the item to the quantity of item for every session.

Original link [here](https://redis.com/ebook/part-1-getting-started/chapter-2-anatomy-of-a-redis-web-application/2-2-shopping-carts-in-redis/).

In [1]:
import utils

In [2]:
conn = utils.connect()
conn.ping()

True

In [3]:
def add_to_cart(conn, session, item, count):
    if count <= 0:
        conn.hrem("cart:" + session, item)
    else:
        conn.hset("cart:" + session, item, count)

In [6]:
# To add 2 books to the cart for the session "token123"
add_to_cart(conn, "token123", "books", 2)

In [9]:
# To retrieve the itesm from the cart.
conn.hget("cart:token123", "books")

'2'

In [14]:
# Seems like quitting doesn't really work.
conn.quit()

True

In [ ]:
def clean_full_sessions(conn):
    while not QUIT:
        size = conn.zcard("recent:")
        if size <= LIMIT:
            time.sleep(1)
            continue

        end_index = min(size - LIMIT, 100)

        sessions = conn.zrange("recent:", 0, end_index - 1)

        session_keys = []
        for sess in sessions:
            session_keys.append("viewed:" + sess)
            session_keys.append("cart:" + sess)

        conn.delete(*session_keys)
        conn.hdel("cart:", *sessions)
        conn.zrem("recent:", *sessions)